In [ ]:
!pip install openai
!pip install langchain_openai

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="KEY",
)


completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a smart home assistant. Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL). Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)"},
    {"role": "user", "content": "I want to turn off lights when I am not at home"}
  ]
)



print(completion.choices[0].message)

TAP RULES GENERATING

In [1]:
import os
import subprocess
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL). 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt_template 
    | model 
    | output_parser
)

# Preparing the input data for the chain
input_data = {
    "devices_output": devices_output,
    "user_input": "Start the dishwasher at 10 PM"
}

# Merging devices_output and user input into a single prompt string
final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

# Invoking the chain with the final input
result = chain.invoke(final_input)

print(result)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

PROPERTIES GENERATING

In [ ]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

# Function to list smartthings devices
def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# Create a prompt template
prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Properties from user input. After that, convert it to Linear Temporal Logic (LTL). 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    """
)

# Define output parser and model
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Create the chain
chain = (
    prompt_template 
    | model 
    | output_parser
)

# Function to generate LTL and write to CSV
def generate_and_save_ltl(user_input, csv_filename='ltl_properties.csv'):
    # Preparing the input data for the chain
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    # Merging devices_output and user input into a single prompt string
    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    # Invoking the chain with the final input
    result = chain.invoke(final_input)

    # Write the result to a CSV file
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([user_input, result])

    return result

# Example usage
user_input = "When the motion sensor is active, turn on the switches"
ltl_property = generate_and_save_ltl(user_input)

print(ltl_property)


================== NEW MAIN VERSION ===========================

In [23]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

os.environ['OPENAI_API_KEY'] = "sk-proj-vi58QD74lKpYVlS8jF7NT3BlbkFJ7h6LtRORhb8DNhT5Y2z0"


In [24]:
def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()
print(devices_output)


[
    {
        "deviceId": "7b66afce-2f5a-4bc4-b55f-6b949389f86d",
        "ownerId": "4ab9d1e8-e9e7-9a7f-c6f8-c62ec3ae220c",
        "name": "metering-switch",
        "label": "Aeotec Outlet 1",
        "deviceManufacturerCode": "0086-0103-0060",
        "manufacturerName": "SmartThingsCommunity",
        "deviceModel": "0103-0060",
        "presentationId": "a15343f1-ecbc-3bb5-9087-20f48fee9047",
        "locationId": "0f4f8426-8724-4501-aa02-06f415c7aa14",
        "roomId": "2f26d22c-ae2d-4c70-bc14-3e7e0d2ab250",
        "sharedLocations": [],
        "components": [
            {
                "id": "main",
                "label": "main",
                "capabilities": [
                    {
                        "id": "switch",
                        "version": 1
                    },
                    {
                        "id": "powerMeter",
                        "version": 1
                    },
                    {
                        "id": "energyMet

In [30]:
# Prompt templates
trigger_action_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL) for NuSMV model. 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    Generate the rules in Linear Temporal Logic (LTL) for NuSMV model only. No extra explanation.
    """
)

property_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Properties from user input. After that, convert it to Linear Temporal Logic (LTL). 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

trigger_action_chain = (
    trigger_action_prompt_template 
    | model 
    | output_parser
)

property_chain = (
    property_prompt_template 
    | model 
    | output_parser
)


In [31]:
def generate_trigger_action_rule(user_input):
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    result = trigger_action_chain.invoke(final_input)

    return result

user_input = "When the motion sensor detect motion, turn on the Aeotec switch"
trigger_action_rule = generate_trigger_action_rule(user_input)

print(trigger_action_rule)


G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)


In [60]:
def generate_and_save_ltl(user_input, trigger_action_rule, csv_filename='ltl_properties.csv'):
    # Preparing the input data for the chain
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    # Merging devices_output and user input into a single prompt string
    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    # Invoking the chain with the final input
    result = property_chain.invoke(final_input)

    # Write the result to a CSV file with UTF-8 encoding
    with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([trigger_action_rule, result])

    return result

# Example usage
user_input_for_property = "When the motion sensor detects motion, turn on the Centralite switch"
ltl_property = generate_and_save_ltl(user_input_for_property, trigger_action_rule)

print(ltl_property)


### Properties from User Input

1. **Motion Sensor**: The device that detects motion.
    - Device ID: `badd4ded-6183-4fb6-928c-179dcd6d1229`
    - Label: "Motion Sensor"
    - Capability: "motionSensor"

2. **Centralite Switch**: The switch that needs to be turned on.
    - Device ID: `03f87109-2da4-4376-8372-580372d9ded6`
    - Label: "Centralite Outlet 1"
    - Capability: "switch"
    
### Linear Temporal Logic (LTL)

The LTL formula to express that whenever the motion sensor detects motion, the Centralite switch should eventually be turned on can be written as:

\[ \mathbf{G} (\text{motion\_sensor\_active} \rightarrow \mathbf{F} \text{switch\_on\_Centralite\_Outlet\_1}) \]

### Explanation

- **G** (Globally): Specifies that the condition should hold at all times.
- **motion\_sensor\_active**: Represents the event when the motion sensor detects motion.
- **F** (Finally): Specifies that the condition should eventually hold.
- **switch\_on\_Centralite\_Outlet\_1**: Represents the ev

In [61]:
def compare_trigger_action_to_properties(trigger_action, properties_filename='ltl_properties.csv'):
    # Read the properties from the CSV file
    with open(properties_filename, mode='r') as file:
        reader = csv.reader(file)
        properties = [row[1] for row in reader]
    
    # Check if the trigger action is in the properties
    if trigger_action in properties:
        return f"The Trigger-Action rule '{trigger_action}' is FOUND in the properties."
    else:
       # Regenerate the rule
        regenerated_rule = generate_trigger_action_rule(trigger_action)
        return f"The Trigger-Action rule '{trigger_action}' was NOT FOUND in the properties. Regenerated rule: {regenerated_rule}"

result = compare_trigger_action_to_properties(trigger_action_rule)

print(result)


The Trigger-Action rule 'Sure, let's start by generating the Trigger-Action rules and then convert them to Linear Temporal Logic (LTL).

### Trigger-Action Rule
1. **Trigger**: Motion sensor detects motion.
2. **Action**: Turn on the Aeotec switch (Aeotec Outlet 1).

### Linear Temporal Logic (LTL) Representation
The LTL formula that represents this behavior is:

\[ G (\text{motion\_sensor\_motion\_detected} \rightarrow F \text{switch\_on\_Aeotec\_Outlet\_1}) \]

Where:
- \( G \) stands for "Globally," indicating that the rule applies at all times.
- \( \rightarrow \) stands for "implies."
- \( F \) stands for "Finally," indicating that at some point in the future, the action will occur if the trigger is true.

In plain English, this LTL formula means that globally, whenever the motion sensor detects motion, it will eventually cause the Aeotec switch to turn on.

### Detailed Explanation of the Formula
- **Globally (G)**: The rule should always be active.
- **Trigger (motion_sensor_mot

=====================AutoGPT========================

=============================================

In [ ]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# Create the prompt
prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Create a dataset with 100 linear temporal logic formulas for Trigger Action rules based on the {devices_output}")

# Initialize the model and output parser
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Create the chain
chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

# Get the result from the chain
result = chain.invoke(input_data)

# Split the result into individual LTL formulas if necessary (assumes result is a single string with formulas separated by newlines)
ltl_formulas = result.split('\n')

# Write the LTL formulas to a CSV file
csv_file_path = 'ltl_formulas.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['LTL Formula'])  # Write header
    for formula in ltl_formulas:
        csvwriter.writerow([formula])

print(f"LTL formulas have been written to {csv_file_path}")


In [ ]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

def generate_trigger_action_rule(devices_output):
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The devices are in {devices_output}. Create a Trigger-Action rule based on the devices in the SmartThings account.  Use the correct deviceId for the devices."
    )
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "devices_output": devices_output
    }

    result = chain.invoke(input_data)
    return result.strip()  # Assuming the result is a single string for one rule

def generate_ltl_formulas(devices_output):
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The devices are in {devices_output}. Create a dataset with 100 linear temporal logic formulas for Trigger Action rules based on the {devices_output}."
    )
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "devices_output": devices_output
    }

    result = chain.invoke(input_data)
    return result.split('\n')  # Assuming the result is a single string with formulas separated by newlines

# Rules Checking

def check_trigger_action_rule_with_ltl(trigger_action_rule, ltl_formulas):
    """
    Checks if the Trigger-Action rule matches any of the LTL formulas using LLM.

    :param trigger_action_rule: Trigger-Action rule
    :param ltl_formulas: List of LTL formulas
    :return: True if the rule matches any LTL formula, False otherwise
    """
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The following is a Trigger-Action rule and its corresponding LTL formulas. Check if the rule complies with any of the LTL formulas. Respond with 'True' if the rule complies with any formula, otherwise respond with 'False'.\n\nTrigger-Action Rule:\n{trigger_action_rule}\n\nLTL Formulas:\n{ltl_formulas}"
    )
    
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "trigger_action_rule": trigger_action_rule,
        "ltl_formulas": "\n".join(ltl_formulas)
    }

    result = chain.invoke(input_data)
    return result.strip().lower() == 'true'

def save_to_csv(data, file_path, header):
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)  # Write header
        for item in data:
            csvwriter.writerow([item])
    print(f"Data has been written to {file_path}")

# Main loop
devices_output = list_smartthings_devices()
matched = False
iteration = 0

# Generate LTL formulas once
ltl_formulas = generate_ltl_formulas(devices_output)
print(f"Generated LTL Formulas:\n{ltl_formulas}")

while not matched and iteration < 10:  # Limit the iterations to avoid an infinite loop
    print(f"Iteration: {iteration+1}")
    trigger_action_rule = generate_trigger_action_rule(devices_output)
    print(f"Generated Trigger-Action Rule:\n{trigger_action_rule}")

    matched = check_trigger_action_rule_with_ltl(trigger_action_rule, ltl_formulas)

    if matched:
        print("Trigger-Action rule matches the LTL formulas.")
        save_to_csv([trigger_action_rule], 'trigger_action_rule.csv', ['Trigger Action Rule'])
    else:
        print("Trigger-Action rule does not match the LTL formulas. Generating new rule...")
    
    iteration += 1

# Save the LTL formulas to a CSV file
save_to_csv(ltl_formulas, 'ltl_formulas.csv', ['LTL Formula'])


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from time import time

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"


prompt = ChatPromptTemplate.from_template(
    """
    You are a smart home assistant. Create a Trigger-Action rule based on the user input.
        
    """
)


output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    {"request": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

In [ ]:
result = chain.invoke("Turn off all devices when nobody is home")

print(result)

In [ ]:
result = chain.invoke("When the front door is opened, turn on the hallway lights")

print(result)

In [ ]:
# Harper's code
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from time import time

os.environ['OPENAI_API_KEY'] = "key"

import subprocess
from openai import OpenAI

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Your task is generating Trigger-Action Rules for Home Automation")
prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Create some Trigger-Action rules based on the devices in the SmartThings account. Use the correct deviceId for the devices")

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

result = chain.invoke(input_data)

print(result)

NuSMV

In [37]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = "sk-proj-vi58QD74lKpYVlS8jF7NT3BlbkFJ7h6LtRORhb8DNhT5Y2z0"

# User inputs to generate NuSMV models
user_inputs = [
    "Lights should be on when someone is in the room."
]

# Define the prompt template
prompt_template = ChatPromptTemplate.from_template(
    """
    You are an assistant that helps generate NuSMV models. Generate NuSMV rules with Linear Temporal Logic for the following user input: {user_input}
    Generate the NuSMV format only, no explanation.
    """
)

# Define the output parser
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Define the chain
chain = (
    prompt_template 
    | model 
    | output_parser
)

# Function to filter and append NuSMV rules to a model file
def append_rule_to_model(rule, model_path):
    # Remove the triple quotes and ```nusmv tags
    filtered_rule = rule.replace("```nusmv", "").replace("```", "").strip()
    with open(model_path, 'a') as file:
        file.write(f"\n{filtered_rule}\n")

# Path to the model file
model_path = "model.smv"

# Clear the existing model file or create a new one
with open(model_path, 'w') as file:
    file.write("")

# Process each user input and append the generated NuSMV rules to the model file
for user_input in user_inputs:
    input_data = {"user_input": user_input}
    result = chain.invoke(input_data)
    append_rule_to_model(result, model_path)

# Output the contents of the model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print(model_contents)



MODULE main
VAR
  light : boolean;
  someone_in_room : boolean;

INIT
  !light & !someone_in_room

TRANS
  (someone_in_room -> next(light)) &
  (!someone_in_room -> next(!light))

LTLSPEC
  G (someone_in_room -> light)



In [35]:
# Prompt templates
trigger_action_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL) for NuSMV model. 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    Generate the rules in Linear Temporal Logic (LTL) for NuSMV model only. No extra explanation.
    """
)


output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

trigger_action_chain = (
    trigger_action_prompt_template 
    | model 
    | output_parser
)

property_chain = (
    property_prompt_template 
    | model 
    | output_parser
)

def generate_trigger_action_rule(user_input):
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    result = trigger_action_chain.invoke(final_input)

    return result

user_input = "When the motion sensor detect motion, turn on the Aeotec switch"
trigger_action_rule = generate_trigger_action_rule(user_input)

print(trigger_action_rule)


G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)


In [38]:
# Define the devices output (example)
devices_output = """
motion_sensor_active : boolean;
switch_on_Aeotec_Outlet_1 : boolean;
"""

# Prompt templates
trigger_action_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL) for NuSMV model. 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    Generate the rules in Linear Temporal Logic (LTL) for NuSMV model only. No extra explanation.
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

trigger_action_chain = (
    trigger_action_prompt_template 
    | model 
    | output_parser
)

def generate_trigger_action_rule(user_input):
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    final_input = {
        "devices_output": input_data['devices_output'],
        "user_input": input_data['user_input']
    }

    result = trigger_action_chain.invoke(final_input)
    return result

# User input for generating TAP rule
user_input = "When the motion sensor detects motion, turn on the Aeotec switch"
trigger_action_rule = generate_trigger_action_rule(user_input)

# Append the TAP rule to the existing model.smv file
def append_tap_rule_to_model(rule, model_path):
    with open(model_path, 'a') as file:
        file.write(f"\nLTLSPEC {rule}\n")

# Path to the model file
model_path = "model.smv"
append_tap_rule_to_model(trigger_action_rule, model_path)

# Output the contents of the updated model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print("Updated NuSMV Model Contents:\n", model_contents)

# Run the NuSMV model file and capture the output
def run_nusmv_model(model_path):
    try:
        result = subprocess.run(['NuSMV', model_path], capture_output=True, text=True)
        print("NuSMV Output:\n", result.stdout)
        if result.stderr:
            print("NuSMV Errors:\n", result.stderr)
    except FileNotFoundError:
        print("NuSMV is not installed or not found in the system PATH.")

# Run the NuSMV model
run_nusmv_model(model_path)


Updated NuSMV Model Contents:
 
MODULE main
VAR
  light : boolean;
  someone_in_room : boolean;

INIT
  !light & !someone_in_room

TRANS
  (someone_in_room -> next(light)) &
  (!someone_in_room -> next(!light))

LTLSPEC
  G (someone_in_room -> light)

LTLSPEC G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)

NuSMV Output:
 *** This is NuSMV 2.5.4 (compiled on Fri Oct 28 14:13:29 UTC 2011)
*** Enabled addons are: compass 
*** For more information on NuSMV see <http://nusmv.fbk.eu>
*** or email to <nusmv-users@list.fbk.eu>.
*** Please report bugs to <nusmv-users@fbk.eu>

*** Copyright (c) 2010, Fondazione Bruno Kessler

*** This version of NuSMV is linked to the CUDD library version 2.4.1
*** Copyright (c) 1995-2004, Regents of the University of Colorado

*** This version of NuSMV is linked to the MiniSat SAT solver. 
*** See http://www.cs.chalmers.se/Cs/Research/FormalMethods/MiniSat
*** Copyright (c) 2003-2005, Niklas Een, Niklas Sorensson 


NuSMV Errors:
 
TYPE ERROR file mode

TESTING

In [9]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = "sk-proj-vi58QD74lKpYVlS8jF7NT3BlbkFJ7h6LtRORhb8DNhT5Y2z0"

# User inputs to generate NuSMV models
user_inputs = [
    "Start the dishwasher at 10 PM."
]

# Define the prompt template
prompt_template = ChatPromptTemplate.from_template(
    """
    You are an assistant that helps generate NuSMV models. Generate NuSMV rules with Linear Temporal Logic for the following user input: {user_input}
    Generate the NuSMV format only, no explanation.
    """
)

# Define the output parser
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Define the chain
chain = (
    prompt_template 
    | model 
    | output_parser
)

# Function to filter and append NuSMV rules to a model file
def append_rule_to_model(rule, model_path):
    # Remove the triple quotes and ```nusmv tags
    filtered_rule = rule.replace("```nusmv", "").replace("```", "").strip()
    
    # Extract only the relevant sections
    var_section = re.findall(r'VAR\n(.*?)\nASSIGN', filtered_rule, re.DOTALL)
    assign_section = re.findall(r'ASSIGN\n(.*?)\nLTLSPEC', filtered_rule, re.DOTALL)
    ltl_spec = re.findall(r'LTLSPEC\n(.*)', filtered_rule, re.DOTALL)
    
    with open(model_path, 'a') as file:
        if var_section:
            file.write(f"\nVAR\n{var_section[0].strip()}\n")
        if assign_section:
            file.write(f"\nASSIGN\n{assign_section[0].strip()}\n")
        if ltl_spec:
            file.write(f"\nLTLSPEC\n{ltl_spec[0].strip()}\n")

# Path to the model file
model_path = "model.smv"

# Clear the existing model file or create a new one with only the MODULE, VAR, and ASSIGN headers
with open(model_path, 'w') as file:
    file.write("MODULE main\nVAR\nASSIGN\n")

# Process each user input and append the generated NuSMV rules to the model file
for user_input in user_inputs:
    input_data = {"user_input": user_input}
    result = chain.invoke(input_data)
    append_rule_to_model(result, model_path)

# Output the contents of the model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print(model_contents)


MODULE main
VAR
ASSIGN

VAR
dishwasher : boolean;

ASSIGN
init(dishwasher) := FALSE;

LTLSPEC
!(dishwasher) U (time = 22);



MODULE time
VAR
  hours : 0..23;
  minutes : 0..59;

DEFINE
  time := hours + minutes / 60;

ASSIGN
  init(hours) := 0;
  init(minutes) := 0;

  next(hours) := case
    minutes = 59 : (hours + 1) mod 24;
    TRUE : hours;
  esac;

  next(minutes) := (minutes + 1) mod 60;



MODULE main
VAR
  time : time;
  dishwasher : boolean;

ASSIGN
  init(dishwasher) := FALSE;

LTLSPEC
  G (time.hours = 22 -> F (dishwasher));



In [10]:
# Prompt templates
trigger_action_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL) for NuSMV model. 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    Generate the rules in Linear Temporal Logic (LTL) for NuSMV model only. No extra explanation.
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

trigger_action_chain = (
    trigger_action_prompt_template 
    | model 
    | output_parser
)

# property_chain = (
#     property_prompt_template 
#     | model 
#     | output_parser
# )

def generate_trigger_action_rule(user_input):
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    result = trigger_action_chain.invoke(final_input)

    return result

devices_output = """
motion_sensor_active : boolean;
switch_on_Aeotec_Outlet_1 : boolean;
"""

user_input = "Routine: dishwashing at night"
trigger_action_rule = generate_trigger_action_rule(user_input)

print(trigger_action_rule)


G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)


In [11]:
import re
import subprocess

# Function to extract variables from the rules
def extract_variables(rule):
    variables = set(re.findall(r'\b(?!G\b|F\b)\w+\b', rule))
    return variables

# Path to the model file
model_path = "model.smv"

# Clean the TAP rule by removing markdown elements
def clean_tap_rule(rule):
    return rule.replace("```ltl", "").replace("```", "").strip()

# Extract variables from the generated TAP rule
tap_variables = extract_variables(trigger_action_rule)

# Read the existing model file
with open(model_path, 'r') as file:
    model_contents = file.readlines()

# Insert the variable declarations into the model file
var_index = model_contents.index("VAR\n") + 1
for var in tap_variables:
    var_declaration = f"  {var} : boolean;\n"
    if var_declaration not in model_contents:
        model_contents.insert(var_index, var_declaration)
        var_index += 1

# Insert the ASSIGN declarations
assign_index = model_contents.index("ASSIGN\n") + 1
for var in tap_variables:
    assign_declaration = f"  init({var}) := FALSE;\n"
    if assign_declaration not in model_contents:
        model_contents.insert(assign_index, assign_declaration)
        assign_index += 1

# Write the updated model file back to disk
with open(model_path, 'w') as file:
    file.writelines(model_contents)

# Clean the TAP rule
cleaned_tap_rule = clean_tap_rule(trigger_action_rule)

# Append the TAP rule to the existing model.smv file
def append_tap_rule_to_model(rule, model_path):
    with open(model_path, 'a') as file:
        file.write(f"\nLTLSPEC {rule}\n")

append_tap_rule_to_model(cleaned_tap_rule, model_path)

# Output the contents of the updated model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print("Updated NuSMV Model Contents:\n", model_contents)

# Run the NuSMV model file and capture the output
def run_nusmv_model(model_path):
    try:
        result = subprocess.run(['NuSMV', model_path], capture_output=True, text=True)
        print("NuSMV Output:\n", result.stdout)
        if result.stderr:
            print("NuSMV Errors:\n", result.stderr)
    except FileNotFoundError:
        print("NuSMV is not installed or not found in the system PATH.")

# Run the NuSMV model
run_nusmv_model(model_path)


Updated NuSMV Model Contents:
 MODULE main
VAR
  motion_sensor_active : boolean;
  switch_on_Aeotec_Outlet_1 : boolean;
ASSIGN
  init(motion_sensor_active) := FALSE;
  init(switch_on_Aeotec_Outlet_1) := FALSE;

VAR
dishwasher : boolean;

ASSIGN
init(dishwasher) := FALSE;

LTLSPEC
!(dishwasher) U (time = 22);



MODULE time
VAR
  hours : 0..23;
  minutes : 0..59;

DEFINE
  time := hours + minutes / 60;

ASSIGN
  init(hours) := 0;
  init(minutes) := 0;

  next(hours) := case
    minutes = 59 : (hours + 1) mod 24;
    TRUE : hours;
  esac;

  next(minutes) := (minutes + 1) mod 60;



MODULE main
VAR
  time : time;
  dishwasher : boolean;

ASSIGN
  init(dishwasher) := FALSE;

LTLSPEC
  G (time.hours = 22 -> F (dishwasher));

LTLSPEC G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)

NuSMV Output:
 *** This is NuSMV 2.5.4 (compiled on Fri Oct 28 14:13:29 UTC 2011)
*** Enabled addons are: compass 
*** For more information on NuSMV see <http://nusmv.fbk.eu>
*** or email to <nusmv-users@

TEST 2

In [73]:
import pandas as pd

# Load the CSV file
csv_file_path = 'tap_safety_properties.csv'
df = pd.read_csv(csv_file_path)

# Display the column names
print(df.columns)


Index(['TAP_rule', 'Safety_property'], dtype='object')


In [74]:
import os
import pandas as pd
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = "sk-proj-vi58QD74lKpYVlS8jF7NT3BlbkFJ7h6LtRORhb8DNhT5Y2z0"

# Load the CSV file containing the safety properties
csv_file_path = 'tap_safety_properties.csv'
df = pd.read_csv(csv_file_path)

# Replace 'correct_column_name' with the actual column name in your CSV file
user_inputs = df['Safety_property'].tolist()

# Define the prompt template
prompt_template = ChatPromptTemplate.from_template(
    """
    You are an assistant that helps generate NuSMV models. Generate NuSMV rules with Linear Temporal Logic for the following user input: {user_input}
    Generate the NuSMV format only, no explanation.
    """
)

# Define the output parser
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Define the chain
chain = (
    prompt_template 
    | model 
    | output_parser
)

# Function to filter and append NuSMV rules to a model file
def append_rule_to_model(rule, model_path):
    # Remove the triple quotes and ```nusmv tags
    filtered_rule = rule.replace("```nusmv", "").replace("```", "").strip()
    
    # Extract only the relevant sections
    var_section = re.findall(r'VAR\n(.*?)\nASSIGN', filtered_rule, re.DOTALL)
    assign_section = re.findall(r'ASSIGN\n(.*?)\nLTLSPEC', filtered_rule, re.DOTALL)
    ltl_spec = re.findall(r'LTLSPEC\n(.*)', filtered_rule, re.DOTALL)
    
    with open(model_path, 'a') as file:
        if var_section:
            file.write(f"\nVAR\n{var_section[0].strip()}\n")
        if assign_section:
            file.write(f"\nASSIGN\n{assign_section[0].strip()}\n")
        if ltl_spec:
            file.write(f"\nLTLSPEC\n{ltl_spec[0].strip()}\n")

# Path to the model file
model_path = "model.smv"

# Clear the existing model file or create a new one with only the MODULE, VAR, and ASSIGN headers
with open(model_path, 'w') as file:
    file.write("MODULE main\nVAR\nASSIGN\n")

# Process each user input and append the generated NuSMV rules to the model file
for user_input in user_inputs:
    input_data = {"user_input": user_input}
    result = chain.invoke(input_data)
    append_rule_to_model(result, model_path)

# Output the contents of the model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print(model_contents)


MODULE main
VAR
ASSIGN

LTLSPEC
G ((!night) -> !dishwashing) &
  G ((night) -> (X dishwashing))

DEFINE
  night := (current_time >= 20 & current_time < 24);
  
VAR
  current_time : 0..23;

VAR
owner_at_home : boolean;
    alarm_on : boolean;

INIT
    owner_at_home = FALSE &
    alarm_on = TRUE;

ASSIGN
next(owner_at_home) := case
        -- Define conditions for the owner's arrival here
        TRUE : {TRUE, FALSE};
    esac;

    next(alarm_on) := case
        owner_at_home : FALSE;
        TRUE : alarm_on;
    esac;

LTLSPEC
G (owner_at_home -> !alarm_on);

VAR
dishwasher : boolean;

ASSIGN
init(dishwasher) := FALSE;

LTLSPEC
G (time = 23 -> !dishwasher)

LTLSPEC
G (hour = 24 -> !dishwasher);

VAR
fan : boolean;
    needed : boolean;

ASSIGN
init(fan) := FALSE;
    init(needed) := FALSE;

LTLSPEC
G (fan -> needed)

VAR
time : {day, night};
  garage : {open, closed};

ASSIGN
init(time) := day;
  init(garage) := closed;

TRANS
  (time = day & next(time) = night) | 
  (time = night & n

In [78]:
# Prompt templates
trigger_action_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL) for NuSMV model. 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    Generate the rules in Linear Temporal Logic (LTL) for NuSMV model only. No extra explanation.
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

trigger_action_chain = (
    trigger_action_prompt_template 
    | model 
    | output_parser
)

property_chain = (
    property_prompt_template 
    | model 
    | output_parser
)

def generate_trigger_action_rule(user_input):
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    result = trigger_action_chain.invoke(final_input)

    return result

devices_output = """
motion_sensor_active : boolean;
switch_on_Aeotec_Outlet_1 : boolean;
"""

user_input = "When the motion sensor detect motion, turn on the Aeotec switch"
trigger_action_rule = generate_trigger_action_rule(user_input)

print(trigger_action_rule)


G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)


In [80]:
import re
import subprocess

# Function to extract variables from the rules
def extract_variables(rule):
    variables = set(re.findall(r'\b(?!G\b|F\b)\w+\b', rule))
    return variables

# Path to the model file
model_path = "model.smv"

# Clean the TAP rule by removing markdown elements
def clean_tap_rule(rule):
    return rule.replace("```ltl", "").replace("```", "").strip()

# Extract variables from the generated TAP rule
tap_variables = extract_variables(trigger_action_rule)

# Read the existing model file
with open(model_path, 'r') as file:
    model_contents = file.readlines()

# Insert the variable declarations into the model file
var_index = model_contents.index("VAR\n") + 1
for var in tap_variables:
    var_declaration = f"  {var} : boolean;\n"
    if var_declaration not in model_contents:
        model_contents.insert(var_index, var_declaration)
        var_index += 1

# Insert the ASSIGN declarations
assign_index = model_contents.index("ASSIGN\n") + 1
for var in tap_variables:
    assign_declaration = f"  init({var}) := FALSE;\n"
    if assign_declaration not in model_contents:
        model_contents.insert(assign_index, assign_declaration)
        assign_index += 1

# Write the updated model file back to disk
with open(model_path, 'w') as file:
    file.writelines(model_contents)

# Clean the TAP rule
cleaned_tap_rule = clean_tap_rule(trigger_action_rule)

# Append the TAP rule to the existing model.smv file
def append_tap_rule_to_model(rule, model_path):
    with open(model_path, 'a') as file:
        file.write(f"\nLTLSPEC {rule}\n")

append_tap_rule_to_model(cleaned_tap_rule, model_path)

# Output the contents of the updated model file
with open(model_path, 'r') as file:
    model_contents = file.read()
    print("Updated NuSMV Model Contents:\n", model_contents)

# Run the NuSMV model file and capture the output
def run_nusmv_model(model_path):
    try:
        result = subprocess.run(['NuSMV', model_path], capture_output=True, text=True)
        print("NuSMV Output:\n", result.stdout)
        if result.stderr:
            print("NuSMV Errors:\n", result.stderr)
    except FileNotFoundError:
        print("NuSMV is not installed or not found in the system PATH.")

# Run the NuSMV model
run_nusmv_model(model_path)


Updated NuSMV Model Contents:
 MODULE main
VAR
  switch_on_Aeotec_Outlet_1 : boolean;
  motion_sensor_active : boolean;
  init(motion_sensor_active) := FALSE;
  init(switch_on_Aeotec_Outlet_1) := FALSE;
ASSIGN

LTLSPEC G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)

NuSMV Output:
 *** This is NuSMV 2.5.4 (compiled on Fri Oct 28 14:13:29 UTC 2011)
*** Enabled addons are: compass 
*** For more information on NuSMV see <http://nusmv.fbk.eu>
*** or email to <nusmv-users@list.fbk.eu>.
*** Please report bugs to <nusmv-users@fbk.eu>

*** Copyright (c) 2010, Fondazione Bruno Kessler

*** This version of NuSMV is linked to the CUDD library version 2.4.1
*** Copyright (c) 1995-2004, Regents of the University of Colorado

*** This version of NuSMV is linked to the MiniSat SAT solver. 
*** See http://www.cs.chalmers.se/Cs/Research/FormalMethods/MiniSat
*** Copyright (c) 2003-2005, Niklas Een, Niklas Sorensson 


NuSMV Errors:
 
file model.smv: line 5: at token "init": syntax error


file 

In [12]:
import pandas as pd

# Load the larger dataset
df = pd.read_csv('tap_safety_properties.csv')

display(df)


,TAP_rule,Safety_property
0,Start the dishwasher at 10 PM,Routine: dishwashing at night
1,Deactivate the alarm when the owner arrives,Convenience: alarm off when owner arrives
2,Turn the heat up to maximum when temperature d...,Safety: temperature should not exceed 80 degrees
3,Turn off the dishwasher at 12 AM,Energy efficiency: dishwasher off by midnight
4,Turn on the fan when temperature is above 75,Energy efficiency: fan should only run when ne...
5,Turn off the dishwasher at 12 AM,Energy efficiency: dishwasher off by midnight
6,Turn the heat up to maximum when temperature d...,Safety: temperature should not exceed 80 degrees
7,Turn on the fan when temperature is above 75,Energy efficiency: fan should only run when ne...
8,Close the garage door at 11 PM,Security: garage closed at night
9,Turn the heat up to maximum when temperature d...,Safety: temperature should not exceed 80 degrees


In [13]:
from random import choice

def gpt_4o_generate_tap_rules(command):
    # Simulated GPT-4o function to generate TAP rules
    tap_rules = {
        'Turn on the light when motion is detected': 'G (motion_detected -> light_on)',
        'Lock the front door at 10 PM': 'G (time_10_PM -> door_locked)',
        'Unlock the front door all day': 'G (time_all_day -> door_unlocked)',
        'Turn the heat up to maximum when temperature drops below 60': 'G (temp_below_60 -> heat_max)',
        'Turn off all lights when the house is empty': 'G (house_empty -> lights_off)',
        'Open the garage door when the car arrives': 'G (car_arrives -> garage_open)',
        'Play music at 7 AM': 'G (time_7_AM -> music_play)',
        'Turn off the oven when smoke is detected': 'G (smoke_detected -> oven_off)',
        'Turn on the sprinklers at 6 AM': 'G (time_6_AM -> sprinklers_on)',
        'Lock all windows at night': 'G (time_night -> windows_locked)',
        'Dim the lights when watching TV': 'G (watching_TV -> lights_dim)',
        'Turn on the fan when temperature is above 75': 'G (temp_above_75 -> fan_on)',
        'Turn off the heater at 8 AM': 'G (time_8_AM -> heater_off)',
        'Activate the security system at 10 PM': 'G (time_10_PM -> security_on)',
        'Deactivate the alarm when the owner arrives': 'G (owner_arrives -> alarm_off)',
        'Start the coffee maker at 6 AM': 'G (time_6_AM -> coffee_on)',
        'Turn on the outdoor lights when it gets dark': 'G (dark -> outdoor_lights_on)',
        'Turn off the outdoor lights at dawn': 'G (dawn -> outdoor_lights_off)',
        'Close the garage door at 11 PM': 'G (time_11_PM -> garage_closed)',
        'Open the blinds at sunrise': 'G (sunrise -> blinds_open)',
        'Close the blinds at sunset': 'G (sunset -> blinds_closed)',
        'Turn on the humidifier when humidity is low': 'G (humidity_low -> humidifier_on)',
        'Turn off the humidifier when humidity is high': 'G (humidity_high -> humidifier_off)',
        'Turn on the AC when it gets hot': 'G (hot -> AC_on)',
        'Turn off the AC at 10 PM': 'G (time_10_PM -> AC_off)',
        'Activate the garden lights at 7 PM': 'G (time_7_PM -> garden_lights_on)',
        'Deactivate the garden lights at 6 AM': 'G (time_6_AM -> garden_lights_off)',
        'Play news on the smart speaker at 7 AM': 'G (time_7_AM -> news_play)',
        'Stop playing music when leaving the house': 'G (leaving_house -> music_stop)',
        'Activate the doorbell camera when motion is detected': 'G (motion_detected -> camera_on)',
        'Turn off the doorbell camera when the owner is home': 'G (owner_home -> camera_off)',
        'Start the washing machine at 9 AM': 'G (time_9_AM -> washer_on)',
        'Turn off the washing machine at 12 PM': 'G (time_12_PM -> washer_off)',
        'Activate the robot vacuum at 2 PM': 'G (time_2_PM -> vacuum_on)',
        'Deactivate the robot vacuum at 4 PM': 'G (time_4_PM -> vacuum_off)',
        'Start the dishwasher at 10 PM': 'G (time_10_PM -> dishwasher_on)',
        'Turn off the dishwasher at 12 AM': 'G (time_12_AM -> dishwasher_off)',
        'Activate the lawn mower at 8 AM': 'G (time_8_AM -> mower_on)',
        'Deactivate the lawn mower at 10 AM': 'G (time_10_AM -> mower_off)',
        'Start the pool filter at 9 AM': 'G (time_9_AM -> filter_on)',
        'Turn off the pool filter at 5 PM': 'G (time_5_PM -> filter_off)',
        'Turn on the bedroom lights at 7 AM': 'G (time_7_AM -> bedroom_lights_on)',
        'Turn off the bedroom lights at 10 PM': 'G (time_10_PM -> bedroom_lights_off)',
        'Turn on the water heater at 6 AM': 'G (time_6_AM -> water_heater_on)',
        'Turn off the water heater at 10 PM': 'G (time_10_PM -> water_heater_off)',
        'Activate the driveway lights when motion is detected': 'G (motion_detected -> driveway_lights_on)',
        'Deactivate the driveway lights when no motion is detected': 'G (no_motion -> driveway_lights_off)',
        'Start the air purifier at 8 AM': 'G (time_8_AM -> purifier_on)',
        'Turn off the air purifier at 10 PM': 'G (time_10_PM -> purifier_off)'
    }
    return tap_rules.get(command, '')

def nusmv_model_check(ltl_formula, safety_property):
    # Simulated NuSMV model checker function
    unsafe_rules = [
        'G (time_all_day -> door_unlocked)',
        'G (temp_below_60 -> heat_max)',
        'G (time_7_AM -> music_play)',
        'G (car_arrives -> garage_open)',
        'G (time_10_PM -> AC_off)'
    ]
    return ltl_formula not in unsafe_rules


In [14]:
# Test the rules
results = []
for index, row in df.iterrows():
    tap_rule = row['TAP_rule']
    safety_property = row['Safety_property']
    ltl_formula = gpt_4o_generate_tap_rules(tap_rule)
    is_safe = nusmv_model_check(ltl_formula, safety_property)
    results.append({
        'TAP_rule': tap_rule,
        'LTL_formula': ltl_formula,
        'Safety_property': safety_property,
        'Is_safe': is_safe
    })

results_df = pd.DataFrame(results)
display(results_df)


,TAP_rule,LTL_formula,Safety_property,Is_safe
0,Start the dishwasher at 10 PM,G (time_10_PM -> dishwasher_on),Routine: dishwashing at night,True
1,Deactivate the alarm when the owner arrives,G (owner_arrives -> alarm_off),Convenience: alarm off when owner arrives,True
2,Turn the heat up to maximum when temperature d...,G (temp_below_60 -> heat_max),Safety: temperature should not exceed 80 degrees,False
3,Turn off the dishwasher at 12 AM,G (time_12_AM -> dishwasher_off),Energy efficiency: dishwasher off by midnight,True
4,Turn on the fan when temperature is above 75,G (temp_above_75 -> fan_on),Energy efficiency: fan should only run when ne...,True
5,Turn off the dishwasher at 12 AM,G (time_12_AM -> dishwasher_off),Energy efficiency: dishwasher off by midnight,True
6,Turn the heat up to maximum when temperature d...,G (temp_below_60 -> heat_max),Safety: temperature should not exceed 80 degrees,False
7,Turn on the fan when temperature is above 75,G (temp_above_75 -> fan_on),Energy efficiency: fan should only run when ne...,True
8,Close the garage door at 11 PM,G (time_11_PM -> garage_closed),Security: garage closed at night,True
9,Turn the heat up to maximum when temperature d...,G (temp_below_60 -> heat_max),Safety: temperature should not exceed 80 degrees,False
